<a href="https://colab.research.google.com/github/patrickaraujo/Span_ASTE_Demo/blob/main/C%C3%B3pia_de_Span_ASTE_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone https://github.com/chiayewken/Span-ASTE.git
!cd Span-ASTE && git checkout f53ec3c
!cp -a Span-ASTE/* .
!echo boto3==1.16.46 >> requirements.txt
!bash setup.sh

Cloning into 'Span-ASTE'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 194 (delta 55), reused 41 (delta 41), pack-reused 116
Receiving objects: 100% (194/194), 615.10 KiB | 2.41 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Note: switching to 'f53ec3c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f53ec3c Add command-line scoring instructions in README.md
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=b018af14a2166d84bf3e80a51323797bfa5b950fd310c7e8392d4c6bf43bb311
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [8]:
#@title Data Exploration
data_name = "15res" #@param ["14lap", "14res", "15res", "16res"]

import sys
sys.path.append("aste")
from data_utils import Data

path = f"aste/data/triplet_data/{data_name}/train.txt"
data = Data.load_from_full_path(path)

for s in data.sentences[:3]:
    print("tokens:", s.tokens)
    for t in s.triples:
        print("target:", (t.t_start, t.t_end))
        print("opinion:", (t.o_start, t.o_end))
        print("label:", t.label)
    print()

tokens: ['Judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.']
target: (10, 10)
opinion: (9, 9)
label: LabelEnum.negative

tokens: ['We', ',', 'there', 'were', 'four', 'of', 'us', ',', 'arrived', 'at', 'noon', '-', 'the', 'place', 'was', 'empty', '-', 'and', 'the', 'staff', 'acted', 'like', 'we', 'were', 'imposing', 'on', 'them', 'and', 'they', 'were', 'very', 'rude', '.']
target: (19, 19)
opinion: (31, 31)
label: LabelEnum.negative

tokens: ['The', 'food', 'was', 'lousy', '-', 'too', 'sweet', 'or', 'too', 'salty', 'and', 'the', 'portions', 'tiny', '.']
target: (1, 1)
opinion: (3, 3)
label: LabelEnum.negative
target: (1, 1)
opinion: (5, 6)
label: LabelEnum.negative
target: (1, 1)
opinion: (8, 9)
label: LabelEnum.negative
target: (12, 12)
opinion: (13, 13)
label: LabelEnum.negative



In [ ]:
# Download pretrained SpanModel weights
from pathlib import Path
template = "https://github.com/chiayewken/Span-ASTE/releases/download/v1.0.0/{}.tar"
url = template.format(data_name)
model_tar = Path(url).name
model_dir = Path(url).stem

!wget -nc $url
!tar -xf $model_tar

In [ ]:
# Use pretrained SpanModel weights for prediction
import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

def predict_sentence(text: str, model: SpanModel) -> Sentence:
    path_in = "temp_in.txt"
    path_out = "temp_out.txt"
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False, weight=1, id=0)
    data = Data(root=Path(), data_split=SplitEnum.test, sentences=[sent])
    data.save_to_path(path_in)
    model.predict(path_in, path_out)
    data = Data.load_from_full_path(path_out)
    return data.sentences[0]

text = "Did not enjoy the new Windows 8 and touchscreen functions ."
model = SpanModel(save_dir=model_dir, random_seed=0)
sent = predict_sentence(text, model)

for t in sent.triples:
    target = " ".join(sent.tokens[t.t_start:t.t_end+1])
    opinion = " ".join(sent.tokens[t.o_start:t.o_end+1])
    print()
    print(dict(target=target, opinion=opinion, sentiment=t.label))

In [ ]:
# Train SpanModel from scratch
random_seed = 4
path_train = f"aste/data/triplet_data/{data_name}/train.txt"
path_dev = f"aste/data/triplet_data/{data_name}/dev.txt"
save_dir = f"outputs/{data_name}/seed_{random_seed}"

model = SpanModel(save_dir=save_dir, random_seed=random_seed)
model.fit(path_train, path_dev)

In [ ]:
# Evaluate SpanModel F1 Score
import json

path_pred = "pred.txt"
path_test = f"aste/data/triplet_data/{data_name}/test.txt"
model.predict(path_in=path_test, path_out=path_pred)
results = model.score(path_pred, path_test)
print(json.dumps(results, indent=2))